<a href="https://colab.research.google.com/github/HrithikNambiar/COVID-19-Cough-Detection/blob/main/xbow_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd '/content/drive/MyDrive/ComParE2021'

/content/drive/MyDrive/ComParE2021


In [ ]:
ls

activate           dist/                             notebooks/   results/
audeep_workspace/  end2you_files/                    opensmile/   scripts/
augmented_dist/    features/                         openxbow/    smile.log
data/              features_2/                       params.yaml  src/
data_2/            Miniconda3-4.5.4-Linux-x86_64.sh  README.md


In [ ]:
!pip install -U PyYAML
!pip install liac-arff
!pip install -U scikit-learn scipy matplotlib

Feature extraction using open XBoW

In [ ]:
import os, yaml
import pandas as pd

In [ ]:
params = {}
with open('params.yaml') as f:
    p = yaml.load(f, Loader=yaml.FullLoader)
    params = p['XBOW']

conf_attributes_map = {'ComParE_2016': 'n1[65]2[65]c'}

# Paths
os_features_folder = '/content/drive/MyDrive/ComParE2021/features_2/opensmile'
features_folder = '/content/drive/MyDrive/ComParE2021/features_2/openXBoW'  #saved here.

os.makedirs(features_folder, exist_ok=True)

In [ ]:
for dir in os.listdir(os_features_folder):
  print(dir)

ComParE_2016


In [ ]:
# Compute BoAW representations from openSMILE LLDs
for dir in os.listdir(os_features_folder):
    for csize in params['csize']:
        for num_assignments in params['num_assignments']:
            for part in ['train', 'val', 'test']:
                output_dir = os.path.join(features_folder, dir, str(csize), str(num_assignments))
                output_file_boaw = os.path.join(output_dir, f'{part}.arff')
                os.makedirs(output_dir, exist_ok=True)
                if dir in conf_attributes_map:
                    xbow_config = f'-i {os.path.join(os_features_folder, dir, part + "_lld.arff")} -attributes {conf_attributes_map[dir]} -o {output_file_boaw}'
                    if part=='train':
                        xbow_config += f' -standardizeInput -size {csize} -a {num_assignments} -log -B {os.path.join(output_dir, "codebook")}'
                    else:
                        xbow_config += f' -b {os.path.join(output_dir, "codebook")}'
                    os.system('java -Xmx12000m -jar ./openxbow/openXBOW.jar -writeName ' + xbow_config)
                else:
                    print(f'{dir} not mapped to attribute config. Mapped confs: {conf_attributes_map}.')

**SVM+ Compare BoAW**

In [ ]:
!python -m src.svm openXBoW # change FEATURE_BASE in src.svm to ./dist/features

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.6294398907103825
              precision    recall  f1-score   support

    negative       0.84      0.97      0.90       183
    positive       0.70      0.29      0.41        48

    accuracy                           0.83       231
   macro avg       0.77      0.63      0.66       231
weighted avg       0.81      0.83      0.80       231


Confusion Matrix:

[[177   6]
 [ 34  14]]
Generating test predictions for optimised parameters {'estimator': 'LinearSVC(C=0.01, max_iter=10000, random_state=42)', 'estimator__C': '0.01', 'estimator__class_weight': 'None', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'None'}
Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.60724043715847
              precision    recall  f1-score   support

    negative       0.84      0.80      0.82       183
    positive       0.35      0.42      0.38        48

    accuracy                      

**SVM + BoAW + Augmented data**

In [ ]:
!python -m src.svm openXBoW  

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.6434426229508197
              precision    recall  f1-score   support

    negative       0.86      0.79      0.82       183
    positive       0.38      0.50      0.43        48

    accuracy                           0.73       231
   macro avg       0.62      0.64      0.63       231
weighted avg       0.76      0.73      0.74       231


Confusion Matrix:

[[144  39]
 [ 24  24]]
Generating test predictions for optimised parameters {'estimator': "LinearSVC(C=0.001, class_weight='balanced', max_iter=10000, random_state=42)", 'estimator__C': '0.001', 'estimator__class_weight': 'balanced', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'StandardScaler()'}
Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.5922131147540983
              precision    recall  f1-score   support

    negative       0.83      0.93      0.88       183
    positive       0.50      0.25      0.33 

In [ ]:
!python -m src.svm opensmile

generating auDeep features on augmented data

In [ ]:
# !cp data/test/*.wav augmented_dist/wav

In [ ]:
# !cp data/train/*.wav augmented_dist/wav

In [ ]:
# downgrade numpy temporarily to avoid errors with auDeep
!pip install numpy==1.15.0

     |████████████████████████████████| 13.9MB 283kB/s 
ERROR: umap-learn 0.5.1 has requirement numpy>=1.17, but you'll have numpy 1.15.0 which is incompatible.
ERROR: tifffile 2021.3.5 has requirement numpy>=1.15.1, but you'll have numpy 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.15.0 which is incompatible.
ERROR: scipy 1.6.1 has requirement numpy>=1.16.5, but you'll have numpy 1.15.0 which is incompatible.
ERROR: pyerfa 1.7.2 has requirement numpy>=1.16, but you'll have numpy 1.15.0 which is incompatible.
ERROR: pyarrow 3.0.0 has requirement numpy>=1.16.6, but you'll have numpy 1.15.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.0 which is incompatible.
ERROR: pandas 1.1.5 has requirement numpy>=1.15.4, but you'll have numpy 1.15.0 which is incompatible.
ERROR: jaxlib 0.1.62+cuda110 has requirement numpy>=1.16, but you'll have numpy 1.15.0 which is incompatible.

In [ ]:
!pip install auDeep

     |████████████████████████████████| 174kB 7.7MB/s 
     |████████████████████████████████| 411.0MB 24kB/s 
     |████████████████████████████████| 3.9MB 22.5MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 358kB 32.7MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 20.1MB 15.7MB/s 
     |████████████████████████████████| 3.8MB 16.5MB/s 
     |████████████████████████████████| 512kB 43.0MB/s 
     |████████████████████████████████| 317kB 45.2MB/s 
     |████████████████████████████████| 112kB 19.1MB/s 
     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 143kB 31.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=8c199c9c9afa7d9828408afcb8ca79b1cab01b50bb201901240d8e6141054400
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for pyper

In [ ]:
# set in scripts/auDeep.sh
# audio_base= "augmented_dist/"  # use "augmented_dist/" for running on augmented data

!bash scripts/auDeep.sh

audeep preprocess --parser audeep.backend.parsers.compare21.Compare21Parser --basedir dist/ --output audeep_workspace/input/spectrograms/ComParE21-0.08-0.04-128-30.nc --window-width 0.08 --window-overlap 0.04 --fixed-length 8 --center-fixed --clip-below -30 --mel-spectrum 128



[ERROR] audeep - No module named 'audeep.backend.parsers.compare21'
audeep preprocess --parser audeep.backend.parsers.compare21.Compare21Parser --basedir dist/ --output audeep_workspace/input/spectrograms/ComParE21-0.08-0.04-128-45.nc --window-width 0.08 --window-overlap 0.04 --fixed-length 8 --center-fixed --clip-below -45 --mel-spectrum 128



[ERROR] audeep - No module named 'audeep.backend.parsers.compare21'
audeep preprocess --parser audeep.backend.parsers.compare21.Compare21Parser --basedir dist/ --output audeep_workspace/input/spectrograms/ComParE21-0.08-0.04-128-60.nc --window-width 0.08 --window-overlap 0.04 --fixed-length 8 --center-fixed --clip-below -60 --mel-spectrum 128



[ERROR] audeep - No modu

training on auDeep


In [ ]:
!python -m src.svm auDeep

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.610655737704918
              precision    recall  f1-score   support

    negative       0.85      0.72      0.78       183
    positive       0.32      0.50      0.39        48

    accuracy                           0.68       231
   macro avg       0.58      0.61      0.58       231
weighted avg       0.74      0.68      0.70       231


Confusion Matrix:

[[132  51]
 [ 24  24]]
Generating test predictions for optimised parameters {'estimator': "LinearSVC(C=0.1, class_weight='balanced', max_iter=10000, random_state=42)", 'estimator__C': '0.1', 'estimator__class_weight': 'balanced', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'StandardScaler()'}
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/drive/

In [ ]:
!python -m src.svm deepspectrum

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/usr/bin/python3: Error while finding module specification for 'src.svm' (ModuleNotFoundError: No module named 'src')


Convert arff to csv

In [ ]:
import json
import glob
import arff
import os
import pandas as pd

OPENSMILE_FEATURES = '/content/drive/MyDrive/ComParE2021/features/opensmile'
OPENXBOW_FEATURES = '/content/drive/MyDrive/ComParE2021/features/openXBoW'

In [ ]:
if __name__=='__main__':
    opensmile_arffs = sorted(glob.glob(f'{OPENSMILE_FEATURES}/**/*.arff', recursive=True))
    for arff_file in opensmile_arffs:
        with open(arff_file) as f:
            arff_data = arff.load(f)
            df = pd.DataFrame(data=arff_data['data'], columns=[attribute[0] for attribute in arff_data['attributes']])
        target_folder = os.path.dirname(arff_file.replace('opensmile', 'opensmile_csv'))
        os.makedirs(target_folder, exist_ok=True)
        target = os.path.join(target_folder, f'{os.path.splitext(os.path.basename(arff_file))[0]}.csv')
        df.to_csv(target, index=False)

    openxbow_arffs = sorted(glob.glob(f'{OPENXBOW_FEATURES}/**/*.arff', recursive=True))
    for arff_file in openxbow_arffs:
        with open(arff_file) as f:
            arff_data = arff.load(f)
            df = pd.DataFrame(data=arff_data['data'], columns=[attribute[0] for attribute in arff_data['attributes']])
        target_folder = os.path.dirname(arff_file.replace('openXBoW', 'openXBoW_csv'))
        os.makedirs(target_folder, exist_ok=True)
        target = os.path.join(target_folder, f'{os.path.splitext(os.path.basename(arff_file))[0]}.csv')
        df.to_csv(target, index=False)